### Implement selection and sorting of electrons

In [1]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# %matplotlib inline

In [2]:
import skimage.measure
from skimage.util import view_as_blocks
import logging

In [3]:
log = logging.getLogger(__name__)

### Importing the data

We are using the Merged Electron Region data.

In [4]:
event_data = pd.read_csv("output_data/merged_energy_electron_data.csv") # Electron Region Event one

In [5]:
# View the merged data head
event_data.head()

,event,eta,phi,et,position,electron,tau
0,1,0,0,7.23932,3,0,0
1,1,0,1,14.78990,6,0,1
2,1,0,2,8.17547,4,0,1
3,1,0,3,6.89318,2,0,0
4,1,0,4,9.89150,7,0,0


### ET values

In [7]:
# Convert the series into a 14x18 matrix
x_et = np.array(event_data['et'])
matrix_et = np.asmatrix(x_et)

# Resizing into size 14x18
matrix_et.resize((14, 18))

Create a dictionary of index and et values.

In [27]:
index_et = {}

for idx, energy in event_data['et'].iteritems():
    index_et[idx] = energy

### Electron values

In [8]:
# Convert the series into a 14x18 matrix
x_elec = np.array(event_data['electron'])
matrix_electron = np.asmatrix(x_elec)

# Resizing into size 14x18
matrix_electron.resize((14, 18))

### Reduce the dimensions of the matrix using 2x2 regions

We will now reduce the dimensions of our matrix from 14x18 to 7x9 using 2x2 region replacements, i.e., replacing 2x2 regions by only the highest value of ET (energy) with true electrons.

In [9]:
# Configuring the size of sub regions
s = 2

We will now get 2x2 sub matrices as blocks from the electron matrix.

In [32]:
blocks_elec = skimage.util.view_as_blocks(matrix_electron, (s, s))

Reshape each matrix into a list format.

In [37]:
blocks_elec = blocks_elec.reshape(-1,s**2)

Getting blocks for index as well

In [40]:
# Convert the series into a 14x18 matrix
x_idx = np.array(event_data.index.to_list())
matrix_index = np.asmatrix(x_idx)

# Resizing into size 14x18
matrix_index.resize((14, 18))

In [46]:
blocks_idx = skimage.util.view_as_blocks(matrix_index, (s, s))

In [49]:
blocks_idx = blocks_idx.reshape(-1,s**2)

In [162]:
# Initialize empty dictionary to save row and index of energy values in matrix
energy_dict = {}

# Loop inside sub matrix of blocks
for i, sub in enumerate(blocks_elec):
    count = 0 # initially count is 0
    pos_list, idx_list = [], []

    # Loop inside each element of submatrix
    for idx, element in enumerate(sub):

        actual_index = blocks_idx[i][idx]

        if element == 1: # if value of electron is 1
            
            if count < 1:
                log.info("Contains more than one Electron in:", idx)
                # Add value with actual index
                idx_list.append(idx)
                pos_list.append({'actual_index': actual_index})
                
            else:
                log.info("Contains more than one Electron in:", idx)
                # Add value of actual index
                idx_list.append(idx)
                pos_list.append({'actual_index': actual_index})

            # Count increment
            count +=1
            
            # Save them into the dictionary
            energy_dict[i] = [idx_list, pos_list]

        else:
            # Contains no electron True (1)
            if count == 0:
                # Add value of actual index
                
                energy_dict[i] = ['NaN']

                log.info("No electron in row")
        

    # Check if count is greater than or equal to 1        
    if count >= 1:
        log.info("Energy is present in matrix:", i)
    else:
        log.info("No energy in matrix:", i)

In [210]:
# energy_dict

check if multiple values in same submatrix have been included.

In [164]:
# Get list of matrix rows with electron True
matrices_with_energy = [k for k,v in energy_dict.items() if v[0] != 'NaN']

In [176]:
for k,v in energy_dict.items():
    
    if v[0] != 'NaN':
        print(v[1])
#     if v[1]:
#         print(v[1])

[{'actual_index': 34}]
[{'actual_index': 39}]
[{'actual_index': 40}]
[{'actual_index': 63}]
[{'actual_index': 49}]
[{'actual_index': 92}]
[{'actual_index': 77}]
[{'actual_index': 98}, {'actual_index': 99}]
[{'actual_index': 85}, {'actual_index': 102}]
[{'actual_index': 109}]
[{'actual_index': 131}]
[{'actual_index': 133}]
[{'actual_index': 134}]
[{'actual_index': 121}]
[{'actual_index': 155}, {'actual_index': 173}]
[{'actual_index': 160}]
[{'actual_index': 188}, {'actual_index': 207}]
[{'actual_index': 213}]
[{'actual_index': 219}, {'actual_index': 236}]
[{'actual_index': 222}]
[{'actual_index': 250}]


We will now get 2x2 sub matrices as blocks from the energy (et) matrix.

In [177]:
blocks_energy = view_as_blocks(matrix_et, (s,s)).reshape(-1,s**2)

In [192]:
final_energy = []

# Loop within submatrix of 2x2 energy value blocks
for row, sub in enumerate(blocks_energy):
    # Check if row contains electron True values
    if row in matrices_with_energy:
        log.info("Contains energy values in row:", row)
        
        # Get the value of index 
        index = np.argmax(sub[energy_dict[row][0]])
        
        # Append the tuple (max energy, actual index (18x14))
        # energy_dict[row][1][index] is the value of dictionary list of actual index 
        # that was the maximum from np argmax
        final_energy.append([sub[energy_dict[row][0]].max(), energy_dict[row][1][index]])
    else:
        final_energy.append([0, energy_dict[row]])

In [193]:
final_energy

[[0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [12.8206, {'actual_index': 34}],
 [0, ['NaN']],
 [0.0, {'actual_index': 39}],
 [16.15002, {'actual_index': 40}],
 [0, ['NaN']],
 [13.3519, {'actual_index': 63}],
 [0, ['NaN']],
 [64.3284, {'actual_index': 49}],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [8.14362, {'actual_index': 92}],
 [10.5057, {'actual_index': 77}],
 [0, ['NaN']],
 [13.4624, {'actual_index': 98}],
 [0, ['NaN']],
 [13.7001, {'actual_index': 85}],
 [0, ['NaN']],
 [0, ['NaN']],
 [11.288, {'actual_index': 109}],
 [0, ['NaN']],
 [12.3744, {'actual_index': 131}],
 [8.79125, {'actual_index': 133}],
 [9.65047, {'actual_index': 134}],
 [0, ['NaN']],
 [5.67157, {'actual_index': 121}],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [0, ['NaN']],
 [12.0682, {'actual_index': 173}],
 [0, ['NaN']],
 [0, ['NaN']],
 [9.62609, {'actual_index': 160}],
 [0, ['NaN']],
 [0

In [194]:
len(final_energy)

63

Saving the values of energy.

In [208]:
energy_vals = [vals[0] for vals in final_energy]

In [209]:
energy_vals

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12.8206,
 0,
 0.0,
 16.15002,
 0,
 13.3519,
 0,
 64.3284,
 0,
 0,
 0,
 8.14362,
 10.5057,
 0,
 13.4624,
 0,
 13.7001,
 0,
 0,
 11.288,
 0,
 12.3744,
 8.79125,
 9.65047,
 0,
 5.67157,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12.0682,
 0,
 0,
 9.62609,
 0,
 0,
 0,
 0,
 11.2701,
 0,
 0,
 12.4474,
 0,
 0,
 15.3848,
 0,
 4.03492,
 0,
 0,
 0,
 0,
 12.1941]

In [203]:
# for vals in final_energy:
#     print(vals[1])
    
#     ## Get actual_index from this 

['NaN']
['NaN']
['NaN']
['NaN']
['NaN']
['NaN']
['NaN']
['NaN']
{'actual_index': 34}
['NaN']
{'actual_index': 39}
{'actual_index': 40}
['NaN']
{'actual_index': 63}
['NaN']
{'actual_index': 49}
['NaN']
['NaN']
['NaN']
{'actual_index': 92}
{'actual_index': 77}
['NaN']
{'actual_index': 98}
['NaN']
{'actual_index': 85}
['NaN']
['NaN']
{'actual_index': 109}
['NaN']
{'actual_index': 131}
{'actual_index': 133}
{'actual_index': 134}
['NaN']
{'actual_index': 121}
['NaN']
['NaN']
['NaN']
['NaN']
['NaN']
['NaN']
['NaN']
{'actual_index': 173}
['NaN']
['NaN']
{'actual_index': 160}
['NaN']
['NaN']
['NaN']
['NaN']
{'actual_index': 188}
['NaN']
['NaN']
{'actual_index': 213}
['NaN']
['NaN']
{'actual_index': 219}
['NaN']
{'actual_index': 222}
['NaN']
['NaN']
['NaN']
['NaN']
{'actual_index': 250}


Save them into a final array.

Reshape them as 7x9

Save their index as well

sort them based on energy values into 6 electrons.

### Get 6 electrons

- Sort them based on their energies

To do: 
Get time complexity